In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm
import gtsam
from gtsam.symbol_shorthand import P, X
from art_skills import ExpressionFactorPoint2, ExpressionDouble, ExpressionPoint2, DebugExpressionPoint2, DebugExpressionFactorPoint2
from art_skills import SlnStrokeExpression2
import loader
import initialize_utils
import utils
from utils import Double_, Point2_, Time
from fit_types import OptimizationLoggingParams, Solution
import plotting2
import sln_fit

%load_ext autoreload
%autoreload 2
%aimport -numpy
%aimport -matplotlib
%aimport -tqdm
%aimport -gtsam

# Setup

In [ ]:
# Constants
noise_model = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)
# Data
letter_data = loader.load_segments('D', index=None)
trajectory_data = letter_data[0]
stroke_data = trajectory_data[0]

In [ ]:
# Debugging
stroke = SlnStrokeExpression2(P(0))
def create_debug_expression_cb(stroke, t, x, y, only_on_failure=True):
    def debug_expression_cb(values, ret):
        # if any(np.isnan(ret)):
        #     print(f'(t, x, y) = ({t:.3f}, {x:.2f}, {y:.2f}), pred = {ret}, P = {values.atVector(P(0)).tolist()}, X = {values.atVector(X(0)).tolist()}')
        if any (np.isnan(stroke.pos(t, x0=values.atPoint2(X(0)), values=values))):
            print(stroke.pos(Double_(t), values.atPoint2(X(0)), 1e-2).value(values))
    return debug_expression_cb

# Fitting

In [ ]:
sol = sln_fit.fit_stroke(stroke_data)

In [ ]:
plotting2.plot_stroke(None, sol)

In [ ]:
plotting2.plot_trajectory(None, sln_fit.fit_trajectory(trajectory_data))

In [ ]:
with Time('fit letter'):
    sol = sln_fit.fit_letter(letter_data)
with Time('plot'):
    plotting2.plot_letter(None, sol)

In [ ]:
params = sln_fit.FitStrokeParams(
    lm_params=utils.create_params(absoluteErrorTol=0, relativeErrorTol=0))
fig, axes = plt.subplots(1, 5, figsize=(25, 5))
all_sols = []
with Time('Fit letters'):
    for letter in 'ABCDE':
        letter_data = loader.load_segments(letter, index=None)
        all_sols.append(sln_fit.fit_letter(letter_data, fit_params=params))
        break
with Time('Plotting'):
    for sol, ax in zip(all_sols, axes):
        plotting2.plot_letter(ax, sol)